Instalamos las librerias necesarias (Python 3.13.7)

In [1]:
!pip install openai-whisper librosa pydub scikit-learn speechrecognition ipykernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 4.3 MB/s  0:00:07m0:00:0100:01


Importamos las librerias necesarias y creamos las funciones que necesitamos para el proyecto

In [4]:
import whisper
import librosa
import numpy as np
from pydub import AudioSegment
import os
from sklearn.cluster import AgglomerativeClustering
import speech_recognition as sr

def convert_mp3_to_wav(mp3_path):
    """Convierte MP3 a WAV manteniendo la calidad"""
    wav_path = mp3_path.replace('.mp3', '_converted.wav')
    audio = AudioSegment.from_mp3(mp3_path)
    # Convertir a mono y 16kHz para mejor procesamiento
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export(wav_path, format='wav')
    return wav_path

def extract_audio_features(audio_path):
    """Extrae características del audio para clustering de hablantes"""
    y, sr = librosa.load(audio_path, sr=16000)
    
    # Extraer MFCCs (características comunes para identificación de voz)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
    
    # Combinar características
    features = np.vstack([mfcc, mfcc_delta, mfcc_delta2])
    return features.T, sr

def simple_speaker_diarization(audio_path, num_speakers=2):
    """Diarización simple basada en características de audio"""
    features, sr = extract_audio_features(audio_path)
    
    # Usar clustering jerárquico para identificar hablantes
    clustering = AgglomerativeClustering(n_clusters=num_speakers)
    labels = clustering.fit_predict(features)
    
    return labels

def transcribe_with_speakers(audio_path):
    """Transcripción con identificación básica de hablantes"""
    
    # 1. Convertir a WAV si es MP3
    if audio_path.endswith('.mp3'):
        print("Convirtiendo MP3 a WAV...")
        audio_path = convert_mp3_to_wav(audio_path)
    
    # 2. Cargar modelo ligero de Whisper (optimizado para CPU)
    print("Cargando modelo Whisper (base)...")
    model = whisper.load_model("base")  # Usar 'base' para CPU
    
    # 3. Transcribir el audio completo
    print("Transcribiendo audio...")
    result = model.transcribe(audio_path)
    
    # 4. Procesar segmentos y agrupar por hablantes aproximados
    print("Procesando segmentos por hablantes...")
    
    # Agrupar segmentos consecutivos por similitud temporal
    segments = result["segments"]
    speaker_segments = []
    current_speaker = "Persona_01"
    
    for i, segment in enumerate(segments):
        if i == 0:
            # Primer segmento
            speaker_segments.append({
                "speaker": current_speaker,
                "text": segment["text"],
                "start": segment["start"],
                "end": segment["end"]
            })
        else:
            # Cambiar de hablante si hay una pausa significativa
            prev_segment = segments[i-1]
            pause_duration = segment["start"] - prev_segment["end"]
            
            if pause_duration > 2.0:  # Pausa de más de 2 segundos
                current_speaker = "Persona_02" if current_speaker == "Persona_01" else "Persona_01"
            
            speaker_segments.append({
                "speaker": current_speaker,
                "text": segment["text"],
                "start": segment["start"],
                "end": segment["end"]
            })
    
    return speaker_segments

def save_transcription(segments, output_file="transcripcion_reunion.txt"):
    """Guarda la transcripción en formato legible"""
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("TRANSCRIPCIÓN DE REUNIÓN - EQUIPO DE SEGURIDAD E INFRAESTRUCTURA\n")
        f.write("=" * 70 + "\n\n")
        
        for segment in segments:
            f.write(f"{segment['speaker']} ({segment['start']:.1f}s - {segment['end']:.1f}s):\n")
            f.write(f"  {segment['text']}\n\n")
    
    print(f"Transcripción guardada en: {output_file}")

# Script principal
if __name__ == "__main__":
    # Configuración
    audio_file = "ejemplo.mp3"  # Cambia por tu archivo
    output_file = "transcripcion_reunion.txt"
    
    try:
        print("Iniciando procesamiento de audio...")
        
        # Procesar audio y transcribir
        segments = transcribe_with_speakers(audio_file)
        
        # Guardar resultados
        save_transcription(segments, output_file)
        
        # Mostrar estadísticas
        total_segments = len(segments)
        persona1_segments = len([s for s in segments if s["speaker"] == "Persona_01"])
        persona2_segments = total_segments - persona1_segments
        
        print(f"\n--- ESTADÍSTICAS ---")
        print(f"Total de segmentos: {total_segments}")
        print(f"Segmentos Persona_01: {persona1_segments}")
        print(f"Segmentos Persona_02: {persona2_segments}")
        print(f"Duración total: {segments[-1]['end']:.1f} segundos")
        
    except Exception as e:
        print(f"Error durante el procesamiento: {e}")

Iniciando procesamiento de audio...
Convirtiendo MP3 a WAV...
Cargando modelo Whisper (base)...
Transcribiendo audio...
Procesando segmentos por hablantes...
Transcripción guardada en: transcripcion_reunion.txt

--- ESTADÍSTICAS ---
Total de segmentos: 40
Segmentos Persona_01: 40
Segmentos Persona_02: 0
Duración total: 122.0 segundos
